In [1]:
import darknet
import cv2
import os
import re
import imageio

configPath = "/storage/gs/rat-emotion/darknet/cfg/2020-06-02_yolo-obj.cfg"
weightPath = "/storage/gs/rat-emotion/darknet/weights/2020-06-07/2020-06-02_yolo-obj_7000.weights"
video = "/storage/gs/rat-emotion/videos/24fps/rat8-control2.mp4"
metaPath = "/storage/gs/rat-emotion/darknet/labeled_data/2020-06-07/obj.data"

net = darknet.load_net_custom(configPath.encode("ascii"), weightPath.encode("ascii"), 0, 1)
meta = darknet.load_meta(metaPath.encode("ascii"))
net_width, net_height = (darknet.network_width(net), darknet.network_height(net))

cap = cv2.VideoCapture(video)

In [ ]:
'''
Author: Andre Telfer
Email: telfer006@gmail.com
'''

import cv2
import os
import numpy as np
import math
import logging
import sys
from pathlib import Path

logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

def sample_frames_from_video(video, n_frames, output_dir, output_prefix='frame', output_ext='png'):
    '''
    Uniform sample frames from video
    @param path/str video: the video to read from
    @param int n_frames: the number of frames to collect from each video
    @param path/str output_dir: the directory to put the frames in
    @param str output_prefix: the prefix for the saved frame images
    @param str output_ext: the extension for the saved frame images
    '''
    
    if not os.path.exists(video):
        logging.critical(f'Video does not exist: {video}')
        return
    
    if type(output_dir) != Path:
        output_dir = Path(output_dir)
        
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    logging.info(f'Processing {video}')
    video_head, video_name = os.path.split(video)
    cap = cv2.VideoCapture(str(video))
    frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    selected_indices = np.random.choice(np.arange(frames+1), n_frames).astype(np.int64)
    selected_indices = list(sorted(selected_indices))
    logging.info(f'Selected frames: {selected_indices}')
    for index in selected_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        
        if not ret:
            logging.error(f"Error fetching frame {index}, skipping")
            continue
        
        # Pad the frame index, this is the standard format for DLC extracted images
        padded_index = str(int(index)).zfill(math.ceil(np.log(frames)/np.log(10)))
        
        output_filepath = output_dir / f"{output_prefix}{padded_index}.{output_ext}"
        cv2.imwrite(str(output_filepath), frame)
        logging.info(f"Wrote frame {index} to {output_filepath}")

def sample_frames_from_videos(videos, n_frames, output_dir, output_prefix='frame', output_ext='png', flat=False):
    '''
    Uniform sample frames from video
    @param path/str video: the videos to read from
    @param int n_frames: the number of frames to collect from each video
    @param path/str output_dir: the directory to put the frames in
    @param str output_prefix: the prefix for the saved frame images
    @param str output_ext: the extension for the saved frame images
    @param bool flat: if flat, then all frames are saved to the same folder and given an output prefix based on their video
    '''
    if type(output_dir) != Path:
        output_dir = Path(output_dir)
    
    for video in videos:
        video_output_dir = output_dir
        video_output_prefix = output_prefix
        
        head, tail = os.path.split(video)
        name, ext =  os.path.splitext(tail)
        if not flat:
            video_output_dir /= name
        else:
            video_output_prefix = f'{name}_{output_prefix}'
            
        sample_frames_from_video(
            video, 
            n_frames, 
            output_prefix=video_output_prefix, 
            output_ext=output_ext, 
            output_dir=video_output_dir
        )